# Match tensor
## NOTE: no where close to fully functional.

In [18]:
import json
import os
import sys
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.layers import merge, Dense, Input,Dropout, Embedding, LSTM, Bidirectional, Activation
from keras.layers import Conv2D
from keras.layers.merge import dot, multiply, add, concatenate
from keras.layers import Merge
from keras.layers.core import Lambda,Reshape, Flatten
from keras.layers.pooling import GlobalMaxPooling2D
from keras.models import Model
from keras.backend import transpose,batch_dot,expand_dims
from HomeDepotCSVReader import HomeDepotReader
import Utilities
from DataPreprocessing import DataPreprocessing
from Feature_Word2Vec import Feature_Word2Vec
from AutomaticQueryExpansion import Word2VecQueryExpansion

In [19]:
# train_filename = '../data/train_play.csv'
# test_filename = '../data/test_play.csv'
# attribute_filename = '../data/attributes_play.csv'
# description_filename = '../data/product_descriptions_play.csv'
# word2vec_model_path='model/word2vec_play.model'
# vocab_path='model/word2vec_play_vocab.json'
# embeddings_path='model/embeddings_play.npz'

train_filename = '../data/train.csv'
test_filename = '../data/test.csv'
attribute_filename = '../data/attributes.csv'
description_filename = '../data/product_descriptions.csv'
word2vec_model_path='model/word2vec.model'
vocab_path='model/word2vec_vocab.json'
embeddings_path='model/embeddings.npz'

In [20]:
reader = HomeDepotReader()

train_query_df, product_df, attribute_df, test_query_df = reader.getQueryProductAttributeDataFrame(train_filename,
                                              test_filename,
                                              attribute_filename,
                                              description_filename)
print("train_query_df:",list(train_query_df))
print("product_df:", list(product_df))
print("attribute_df:", list(attribute_df))
print("test_query_df:", list(test_query_df))

===========Tranforming labels...
showing current values
trainDF: ['id', 'product_uid', 'search_term', 'relevance']
trainDF:    id  product_uid    search_term  relevance
0   2       100001  angle bracket        3.0
self.mergedLabelDF: ['relevance'] 
 <class 'pandas.core.frame.DataFrame'> (74067, 1)    relevance
0        3.0
Old unique Labels: [ 1.    1.25  1.33  1.5   1.67  1.75  2.    2.25  2.33  2.5   2.67  2.75
  3.  ]
newLabels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Creating new column for training:  relevance_int
===========Transform labels completed
train_query_df: ['id', 'product_uid', 'search_term', 'relevance', 'relevance_int', 'product_idx']
product_df: ['product_title', 'product_uid', 'product_description']
attribute_df: ['product_uid', 'name', 'value']
test_query_df: ['id', 'product_uid', 'search_term']


In [21]:
#transform attribute into doc
dp = DataPreprocessing()
attribute_doc_df = dp.getAttributeDoc(attribute_df)
#attribute_doc_df
product_df=product_df.join(attribute_doc_df.set_index('product_uid'), on = 'product_uid')

In [22]:
# product_df['content'] = train_query_df['search_term'].map(str) + " " + \
#                         product_df['product_title'].map(str) + " " + \
#                         product_df['product_description'].map(str) + " " + \
#                         product_df['attr_json'].map(str)

product_df['content'] = product_df['product_title'].map(str) + " " + \
                        product_df['product_description'].map(str) 

# ## no attribute
# product_df['content'] = product_df['product_title'].map(str) + " " + \
#                         product_df['product_description'].map(str) 
        
#product_df['content']

In [23]:
w2v=Feature_Word2Vec(modelFilename=word2vec_model_path)#modelFilename=word2vec_model_path
# sentences=w2v.convertDFIntoSentences(product_df,'content')
# print(sentences)
# w2v.trainModel(sentences)

Loading Word2Vec model


In [24]:
len(w2v.model.wv.vocab)

90160

In [25]:
w2vExpand=Word2VecQueryExpansion(modelFilename=word2vec_model_path)
query="cooking"
print("Expanding query: ")
print(w2vExpand.getExpandedQuery(query,maxNoOfAdditionalWords=2,minSimilarityLevel=0.65,))

Expanding query: 


AttributeError: 'NoneType' object has no attribute 'getSimilarWordVectors'

In [26]:
print(w2v.getVectorFromWord('wood'))
print(w2v.getSimilarWordVectors('wood',5))
print(len(w2v.getVectorFromWord('wood')))

[-2.43476653 -2.12013912 -3.50656533  0.58745241  4.90255594 -1.57691193
  1.78680015 -0.11155301 -0.8516894   1.42515826 -2.75596309  0.18071532
 -1.38263702 -0.47668794 -0.59346086 -6.05967808 -0.97823668 -1.67771959
 -1.73966205 -2.70276213 -2.4495852   1.80111849 -0.89905769 -1.40062082
  3.26698637  0.81907994  0.87804192 -1.51375568 -0.68455589  0.89254838
 -1.89753783 -2.33326006  4.39501572 -2.62253141  3.26578355 -1.13265657
  2.79018235 -1.45624065  2.09555626  1.32436407  0.95695877 -0.91072303
 -1.58147895 -0.53234851 -1.25787449  2.60771489  2.89738655 -1.63351154
 -2.24115086 -2.93693399 -4.12886047 -0.95569009  3.5763886   3.87186575
 -1.88895512  0.59957284  2.71433902 -1.24780202  2.58502841  5.36405277
  3.1656065   0.80844098  3.32259512 -0.61373973  1.88472426  2.75964165
 -0.11635986  0.55207729  0.46152747 -2.64907169  1.99799168 -1.68025887
  2.06911063 -3.60249233 -1.34680748  1.31298947  0.91403908  2.75914049
  0.65266073 -2.20277786 -0.46578333 -1.56346524  0

# embeddings to keras
http://ben.bolte.cc/resources/embeddings/embeddings.html

In [27]:
vocab = dict([(k, v.index) for k, v in w2v.model.wv.vocab.items()])
with open(vocab_path, 'w') as f:
    f.write(json.dumps(vocab))

In [28]:
weights = w2v.model.wv.syn0
np.save(open(embeddings_path, 'wb'), weights)

In [29]:
def load_vocab(vocab_path):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [30]:
word2idx, idx2word = load_vocab(vocab_path)

In [31]:
def word2vec_embedding_layer(embeddings_path):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """

    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights])
    return layer

In [32]:
print_tokens=False

In [33]:
# variable arguments are passed to gensim's word2vec model
# if options.train:
#     print('Training Word2Vec...')
#     create_embeddings(options.data, options.embeddings, options.vocab, size=100, min_count=5, window=5, sg=1, iter=25)

word2idx, idx2word = load_vocab(vocab_path)

if print_tokens:
    print('Tokens:', ', '.join(word2idx.keys()))



# convert to idx

In [34]:
def query_sent2idx(df,col):
    query_word2vec_idx_list=[]
    queries=w2v.convertDFIntoSentences(df,col)
    print(len(queries))
    for query in queries:
        idx_list = []
        for word in query:
            if word not in word2idx.keys():
                idx_list+=[0]#[len(word2idx.keys())] # use last as special key #TODO: well we need to fix this. Using 0 for now so it's in range To OOV or something random
            else:
                idx_list+=[word2idx[word]]
        query_word2vec_idx_list+=[idx_list]
        #print("=====")
        #print(idx_list)
        #print("=====")
    return query_word2vec_idx_list


In [35]:
query_word2vec_idx_list = query_sent2idx(train_query_df,'search_term')
#print(query_word2vec_idx_list)

75575


In [36]:
# def doc_sent2idx(df,col):
#     doc_word2vec_idx_list=[]
#     for row in df.iteritems():
#         a=w2v.convertDFIntoSentences(row,col)
#         print(a)
# #    print(len(queries))
# #     for query in queries:
# #         idx_list = []
# #         for word in query:
# #             if word not in word2idx.keys():
# #                 idx_list+=[len(word2idx.keys())] # use last as special key
# #             else:
# #                 idx_list+=[word2idx[word]]
# #         query_word2vec_idx_list+=[idx_list]
# #         print("=====")
# #         print(idx_list)
# #         print("=====")
#     return doc_word2vec_idx_list

In [37]:
#TODO: this is fucked. just bodge for testing nn
doc_word2vec_idx_list = query_sent2idx(product_df,'product_title')
#print(doc_word2vec_idx_list)

277395


In [38]:
# # cosine similarity model
# print('Building model...')
# input_a = Input(shape=(1,), dtype='int32', name='input_a')
# input_b = Input(shape=(1,), dtype='int32', name='input_b')
# embeddings = word2vec_embedding_layer(embeddings_path)
# embedding_a = embeddings(input_a)
# embedding_b = embeddings(input_b)
# similarity = merge([embedding_a, embedding_b], mode='cos', dot_axes=2)
# model = Model(input=[input_a, input_b], output=similarity)
# model.compile(optimizer='sgd', loss='mse') # optimizer and loss don't matter


# word_a = 'wood'#raw_input('First word: ')
# if word_a not in word2idx:
#     print('"%s" is not in the index' % word_a)
# word_b = 'fan'#raw_input('Second word: ')
# if word_b not in word2idx:
#     print('"%s" is not in the index' % word_b)
# output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
# print('%f' % output)

# Define model

In [39]:
query_max_len = 6 #covers 95.74% of the search lengths (see data exploration)
doc_max_len = 400 # todo: confirm this is sensible

In [40]:
print('Building model...')

def get_R(x):
    a, b = x.values()
    return K.dot(a, b)


embeddings = word2vec_embedding_layer(embeddings_path)

#embedding lookup
# TODO: replace this with embedding with mask zero rather than padding, need to change index for encoding too, imput dim too.
# TODO: OOV embedding
input_query = Input(shape=(query_max_len,), dtype='int32', name='input_query')
input_doc = Input(shape=(doc_max_len,), dtype='int32', name='input_doc')
input_exact_match = Input(shape=(query_max_len,doc_max_len), dtype='float32', name='input_exact_match')
embedding_query = embeddings(input_query)  # (None, 6, 100)
embedding_doc = embeddings(input_doc)      # (None, 400, 100)

#shared linear projection
shared_lp = Dense(40,activation='linear')
query_output = shared_lp(embedding_query) # (None, 6, 40) 
doc_output = shared_lp(embedding_doc) #(None, 400, 40) 

#query: bi LSTM, lp -- implementation = 0 for CPU option, 1 or 2 for GPU
query_output = Bidirectional(LSTM(30, dropout=0.5, implementation=0, return_sequences=True, go_backwards=True)\
                             ,merge_mode='mul'#TODO: alts are 'sum','mul','ave','concat'<--default, None
                            )(query_output) #(None, 6, 30) unless concat (None, 6, 60)  
query_output = Dense(50,activation='linear')(query_output) #(None, 50)

#doc: bi LSTM, lp
doc_output = Bidirectional(LSTM(30, dropout=0.5, implementation=0, return_sequences=True, go_backwards=True)\
                             ,merge_mode='mul'#TODO: alts are 'sum','mul','ave','concat'<--default, None
                            )(doc_output) #(None, 6, 400) unless concat (None, 6, 800)  
doc_output = Dense(50,activation='linear')(doc_output) #(None, 50)

#2d product
#mt_input = batch_dot(query_output,doc_output,axes=None) #axes=[2,2])
#query_output = Flatten()(query_output)
#doc_output = Flatten()(doc_output)
#mt_input = multiply([query_output, doc_output])
#mt_input = dot([query_output, doc_output], axes=(0), normalize=False)  
dot_product_output = dot([query_output, doc_output], axes=(2), normalize=True)  
#output is (11, 6, 400), where did 50 go?
def func_expand_dims(x):
    return expand_dims(x, axis=-1)

def expand_dims_output_shape(input_shape):
    return (input_shape[0], input_shape[1],input_shape[2],1)

mt_input_1 = Lambda(func_expand_dims, expand_dims_output_shape)(dot_product_output)

#mt_input = dot([transpose(query_output), transpose(doc_output)], axes=(0), normalize=False)  
######################################

def state_layer_dot_prod(x):
    output_list=[]
#    output = dot([x[0], x[1]], axes=(2), normalize=False)  
    for i in range(50):
        print(i)
        print(x[0][i])
        print(x[1][i])
        #output = multiply([transpose(x[0][i]), x[1][i]])
        #output_list += output
        #output = dot([x[0][i], transpose(x[1][i])])
        #transpose(x[1][i])
    print(output_list)
    return output_list

def state_layer_dot_prod_shape(input_shape):
    'Merge output shape'
    shape = list(input_shape)
    #print(input_shape)
    #print(shape)
    outshape = (shape[0][0],shape[1][1],shape[0][1],shape[0][2])
    #print(outshape)
    return tuple(outshape)

######################################
#mt_input = Lambda(state_layer_dot_prod,output_shape=state_layer_dot_prod_shape)([transpose(query_output), transpose(doc_output)])
#mt_input = Lambda(state_layer_dot_prod,output_shape=state_layer_dot_prod_shape)([query_output, doc_output])

##TODO fix error:
#InvalidArgumentError: Input to reshape is a tensor with 4800 values, but the requested shape requires a multiple of 120000
#[[Node: reshape_23/Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](lambda_40/dot_42/MatMul, reshape_23/Reshape/shape)]]
#mt_input = Reshape((doc_max_len, query_max_len,50))(mt_input)

#Append exact match channel
output_exact_match = Dense(400,activation='linear')(input_exact_match)
mt_input_2 = Lambda(func_expand_dims, expand_dims_output_shape)(output_exact_match)
# mt_input = concatenate([mt_input_1,mt_input_2],axis=0)
mt_input = concatenate([mt_input_1,mt_input_2],axis=3)

# Conv layers 1
output1 = Conv2D(filters=18, kernel_size=(3, 3), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

output2 = Conv2D(filters=18, kernel_size=(3, 4), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

output3 = Conv2D(filters=18, kernel_size=(3, 5), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

# Conv layers 2
output1 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output1)

output2 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output2)

output3 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
                dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output3)

# Max pool layer
output1 = GlobalMaxPooling2D(data_format='channels_last')(output1)
output2 = GlobalMaxPooling2D(data_format='channels_last')(output2)
output3 = GlobalMaxPooling2D(data_format='channels_last')(output3)

# Merge
output_combined = add([output1,output2,output3])

# Final layer
output = Dense(1,activation='sigmoid')(output_combined)


# build model
#model = Model([input_query], [query_output])
#model = Model([input_doc], [encoded_doc])
#model = Model([input_query,input_doc], [output1,output2,output3])
#model = Model([input_query,input_doc], [output_combined])
#model = Model([input_query,input_doc], [query_output])
#model = Model([input_query,input_doc], [query_output,doc_output])
#model = Model([input_query,input_doc], [output])
#model = Model([input_query,input_doc,input_exact_match], [mt_input])
#model = Model([input_query,input_doc,input_exact_match], [mt_input_1,mt_input_2])
model = Model([input_query,input_doc,input_exact_match], [output])


model.summary()

Building model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_query (InputLayer)         (None, 6)             0                                            
____________________________________________________________________________________________________
input_doc (InputLayer)           (None, 400)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          multiple              9016000                                      
____________________________________________________________________________________________________
dense_1 (Dense)                  multiple              4040                                         
_________________________________________________________________________

In [41]:
# # from keras.layers import Input, merge
# # from keras.models import Model
# # import numpy as np

# # input_a = np.reshape([[1, 2, 3],[1, 2, 3],],[[1, 2, 3],[1, 2, 3]])
# # input_b = np.reshape([4, 5, 6], (1, 1, 3))

# # print(input_a)
# # print(input_b)
# # a = Input(shape=(1, 3))
# # b = Input(shape=(1, 3))

# # concat = merge([a, b], mode='concat', concat_axis=-1)
# # dot = merge([a, b], mode='dot', dot_axes=2)
# # cos = merge([a, b], mode='cos', dot_axes=2)

# # model_concat = Model(input=[a, b], output=concat)
# # model_dot = Model(input=[a, b], output=dot)
# # model_cos = Model(input=[a, b], output=cos)

# # print(model_concat.predict([input_a, input_b]))
# # print(model_dot.predict([input_a, input_b]))
# # print(model_cos.predict([input_a, input_b]))

# from keras import backend as K
# x_batch = K.ones(shape=(11, 6,50 ))
# y_batch = K.ones(shape=(11, 400,50))
# x_batch=K.batch_flatten(x_batch)
# y_batch=K.batch_flatten(y_batch)
# #xy_batch_dot = K.dot(transpose(y_batch),x_batch )#, axes=[0,0])
# xy_batch_dot = K.batch_dot(x_batch,y_batch, axes=[0,0])
# K.int_shape(xy_batch_dot)
# # K.int_shape(x_batch)
# # K.int_shape(y_batch)



In [42]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [50]:
x_query_train=np.array(query_word2vec_idx_list)[0:74067]
x_doc_train=np.array(doc_word2vec_idx_list)[0:74067]#[0:len(x_query_train)]

# TODO: replace this with embedding with mask zero rather than padding, need to change index for encoding too, imput dim too.
print("Pad sequences (samples x time)")
x_query_train = sequence.pad_sequences(x_query_train, maxlen=query_max_len)
x_doc_train = sequence.pad_sequences(x_doc_train, maxlen=doc_max_len)
# x_query_test = sequence.pad_sequences(x_query_test, maxlen=query_max_len)
# x_doc_test = sequence.pad_sequences(x_doc_test, maxlen=doc_max_len)

print('x_query_train shape:', x_query_train.shape)
print('x_doc_train shape:', x_doc_train.shape)
# print('x_query_test shape:', x_query_test.shape)
# print('x_doc_test shape:', x_doc_test.shape)

#TODO: y label on 0-1 scale
y_train=train_query_df['relevance_int'].as_matrix()
# y_test=test_query_df['relevance_int'].as_matrix()
print('y_train shape:', y_train.shape)

Pad sequences (samples x time)
x_query_train shape: (74067, 6)
x_doc_train shape: (74067, 400)
y_train shape: (74067,)


In [51]:
exact_match_list=[]
for i in range(x_query_train.shape[0]):
    #print(i)
    c1=x_query_train[i]
    c2=x_doc_train[i]
    #print(c1.shape[0])
    #print(c2.shape[0])

    c1_inp=np.repeat(c1,c2.shape[0],axis=0)
    c1_inp=c1_inp.reshape((c1.shape[0],c2.shape[0]))
    #print(c1_inp.shape)
    #print(c1_inp)

    #print(c2)
    c2_inp=np.tile(c2,(c1.shape[0],1))
    #print(c2_inp.shape)
    #print(c2_inp)    
    #print(c1_inp == c2_inp)
    exact_match_list+=[(c1_inp == c2_inp).astype(int)]

In [52]:
exact_match_inp_train=np.array(exact_match_list)

In [53]:
exact_match_inp_train.shape

(74067, 6, 400)

In [ ]:
exact_match_inp_train[0]

In [ ]:
batch_size=100#2
print('Train...')
hist=model.fit([x_query_train,x_doc_train,exact_match_inp_train], [y_train],
          batch_size=batch_size,
          epochs=3,
          #validation_data=[x_test, y_test])
         )


Train...
Epoch 1/3
74067/74067 [==============================] - 1727s - loss: 63.3991 - acc: 5.4005e-05     

In [ ]:
batch_size=2
prediction=model.predict([x_query_train,x_doc_train,exact_match_inp_train],
          batch_size=batch_size,
         )

In [ ]:
prediction.shape

In [ ]:
prediction

In [ ]:
print("Predict")
word_a = 'wood'#raw_input('First word: ')
if word_a not in word2idx:
    print('"%s" is not in the index' % word_a)
word_b = 'fan'#raw_input('Second word: ')
if word_b not in word2idx:
    print('"%s" is not in the index' % word_b)
output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
print('%f' % output)